<a href="https://colab.research.google.com/github/DimpleB0501/eva8/blob/main/Session_3/Assignment_3b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(torch.nn.Module):
     
  def __init__(self):   
        super(Net, self).__init__()
        # Convolution (In LeNet, 28x28 images are given as input. Hence padding of 2 is done below)
        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5, stride=1, padding=1, bias=False) # changed padding to 0
        self.conv2 = torch.nn.Conv2d(in_channels=8, out_channels=16, kernel_size=5, stride=1, padding=0, bias=False) # changed padding to 0
        # batch norm
        self.bn1   = nn.BatchNorm2d(16)
        # Max-pooling
        self.avg_pool_1 = torch.nn.AvgPool2d(kernel_size=2)

        # Convolution
        self.conv3 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0, bias=False)
        # batch norm
        self.bn2   = nn.BatchNorm2d(32)
        # Max-pooling
        #self.max_pool_2 = torch.nn.MaxPool2d(kernel_size=2)
        self.avg_pool_2 = torch.nn.AvgPool2d(kernel_size=2)

        # GAP
        self.gap = torch.nn.AdaptiveAvgPool2d((1,1))

        self.flatten = torch.nn.Flatten()

        # Fully connected layer
        self.fc1 = torch.nn.Linear(32, 16)   # convert matrix with 16*4*4 (= 256) features to a matrix of 120 features (columns)
        self.fc2 = torch.nn.Linear(16, 10)       # convert matrix with 120 features to a matrix of 84 features (columns)
        #self.fc3 = torch.nn.Linear(30, 10)        # convert matrix with 84 features to a matrix of 10 features (columns)
        self.dropout = nn.Dropout2d(0.1)
        
  def forward(self, x):
        # convolve, then perform ReLU non-linearity
        x = torch.nn.functional.relu(self.conv1(x))
        x = self.dropout(torch.nn.functional.relu(self.bn1(self.conv2(x))))
        # max-pooling with 2x2 grid
        x = self.avg_pool_1(x)
        # convolve, then perform ReLU non-linearity
        x = self.dropout(torch.nn.functional.relu(self.bn2(self.conv3(x))))
        # max-pooling with 2x2 grid
        x = self.avg_pool_2(x)
        x = self.gap(x)

        x = self.flatten(x)

        # FC-1, then perform ReLU non-linearity
        x = torch.nn.functional.relu(self.fc1(x))

        # FC-2
        x = self.fc2(x)
        
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]             200
            Conv2d-2           [-1, 16, 22, 22]           3,200
       BatchNorm2d-3           [-1, 16, 22, 22]              32
         Dropout2d-4           [-1, 16, 22, 22]               0
         AvgPool2d-5           [-1, 16, 11, 11]               0
            Conv2d-6             [-1, 32, 7, 7]          12,800
       BatchNorm2d-7             [-1, 32, 7, 7]              64
         Dropout2d-8             [-1, 32, 7, 7]               0
         AvgPool2d-9             [-1, 32, 3, 3]               0
AdaptiveAvgPool2d-10             [-1, 32, 1, 1]               0
          Flatten-11                   [-1, 32]               0
           Linear-12                   [-1, 16]             528
    

<ipython-input-2-8693a69de36f>:52: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(20):
    print ("\nEpoch:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Epoch: 0


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-2-8693a69de36f>:52: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.18316811323165894 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.35it/s]



Test set: Average loss: 0.1367, Accuracy: 9635/10000 (96.3%)


Epoch: 1


loss=0.202875018119812 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.54it/s]



Test set: Average loss: 0.0710, Accuracy: 9784/10000 (97.8%)


Epoch: 2


...


Test set: Average loss: 0.0247, Accuracy: 9926/10000 (99.3%)


Epoch: 18


loss=0.04026010259985924 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 29.62it/s]



Test set: Average loss: 0.0225, Accuracy: 9926/10000 (99.3%)


Epoch: 19


loss=0.050737157464027405 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.69it/s]



Test set: Average loss: 0.0232, Accuracy: 9927/10000 (99.3%)

